# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

sum = 0
n = 0
for sqrt_value in generator:
    sum += sqrt_value
    n += 1
    print(f'{n} square root: {sqrt_value} \n running total: {sum}' )



1 square root: 1.0 
 running total: 1.0
2 square root: 1.4142135623730951 
 running total: 2.414213562373095
3 square root: 1.7320508075688772 
 running total: 4.146264369941973
4 square root: 2.0 
 running total: 6.146264369941973
5 square root: 2.23606797749979 
 running total: 8.382332347441762
6 square root: 2.449489742783178 
 running total: 10.83182209022494
7 square root: 2.6457513110645907 
 running total: 13.47757340128953
8 square root: 2.8284271247461903 
 running total: 16.30600052603572
9 square root: 3.0 
 running total: 19.30600052603572
10 square root: 3.1622776601683795 
 running total: 22.4682781862041
11 square root: 3.3166247903554 
 running total: 25.7849029765595
12 square root: 3.4641016151377544 
 running total: 29.249004591697254
13 square root: 3.605551275463989 
 running total: 32.854555867161245


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [24]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [25]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='homework')


# we can load any generator to a table at the pipeline destination as follows:
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.22 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework
The duckdb destination used duckdb:////home/lulu/projects/data_talks/de-zoomcamp-2024/3b-dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707744905.5324426 is LOADED and contains no failed jobs


In [26]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))



Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [27]:
# and the data
print("people table below:")

my_people = conn.sql("SELECT * FROM people").df()
display(my_people)

print (f"Sum of Ages {my_people['age'].sum()}")


people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707744905.5324426,QEgq+UpRN8Tdkg,None
1,2,Person_2,27,City_A,1707744905.5324426,zTQJ1pzUC8T8WA,None
2,3,Person_3,28,City_A,1707744905.5324426,ENWu8XcNqi3MAQ,None
3,4,Person_4,29,City_A,1707744905.5324426,ASagvl3706ZAdA,None
4,5,Person_5,30,City_A,1707744905.5324426,kvTSku6d+t46HA,None


Sum of Ages 140


In [29]:
# ADD THE SECOND SET OF PEOPLE 

# we can load the next generator to the same or to a different table.
info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.15 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework
The duckdb destination used duckdb:////home/lulu/projects/data_talks/de-zoomcamp-2024/3b-dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707744926.9822433 is LOADED and contains no failed jobs


In [30]:
# and the data
print("people table below:")

my_people = conn.sql("SELECT * FROM people").df()
display(my_people)

print (f"Sum of Ages {my_people['age'].sum()}")

people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707744905.5324426,QEgq+UpRN8Tdkg,None
1,2,Person_2,27,City_A,1707744905.5324426,zTQJ1pzUC8T8WA,None
2,3,Person_3,28,City_A,1707744905.5324426,ENWu8XcNqi3MAQ,None
3,4,Person_4,29,City_A,1707744905.5324426,ASagvl3706ZAdA,None
4,5,Person_5,30,City_A,1707744905.5324426,kvTSku6d+t46HA,None
5,3,Person_3,33,City_B,1707744926.9822433,bw7emb12AgIVig,Job_3
6,4,Person_4,34,City_B,1707744926.9822433,Cmxt9v0i469IHg,Job_4
7,5,Person_5,35,City_B,1707744926.9822433,7BCY7SOpLeIiVA,Job_5
8,6,Person_6,36,City_B,1707744926.9822433,OLHB1FW+tB8NGg,Job_6
9,7,Person_7,37,City_B,1707744926.9822433,NU+uQEjvdxaM6g,Job_7


Sum of Ages 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [31]:
# CREATE A SECOND TABLE SO I CAN PRACTICE MERGING ON THE KEYS

# load people_1 generator to a new table 
info = generators_pipeline.run(people_1(),
										table_name="people_merge",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.16 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework
The duckdb destination used duckdb:////home/lulu/projects/data_talks/de-zoomcamp-2024/3b-dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707745839.9468408 is LOADED and contains no failed jobs


In [32]:
# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_merge        │
└─────────────────────┘

In [34]:
my_people_merge = conn.sql("SELECT * FROM people_merge").df()
display(my_people_merge)

,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707745839.9468408,udNaWT64U/3iwA
1,2,Person_2,27,City_A,1707745839.9468408,FxfL37eCsBaV4w
2,3,Person_3,28,City_A,1707745839.9468408,BIYZFpxoJzoWCw
3,4,Person_4,29,City_A,1707745839.9468408,BAcgU+YYKuJIWQ
4,5,Person_5,30,City_A,1707745839.9468408,/LKghG4Ba7YE1A


In [38]:
# run the pipeline with default settings, and capture the outcome
info = generators_pipeline.run(people_2(),
					table_name="people_merge",
					write_disposition="merge",
                    primary_key='id')

# show the outcome

my_people_merge = conn.sql("SELECT * FROM people_merge").df()
display(my_people_merge)

print (f"Sum of Ages {my_people_merge['age'].sum()}")


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707745839.9468408,udNaWT64U/3iwA,None
1,2,Person_2,27,City_A,1707745839.9468408,FxfL37eCsBaV4w,None
2,5,Person_5,35,City_B,1707746162.9137092,ELIg68+BM3GCRA,Job_5
3,4,Person_4,34,City_B,1707746162.9137092,inBdLTk+p4nxsA,Job_4
4,3,Person_3,33,City_B,1707746162.9137092,K5QI2KSo1eGq8w,Job_3
5,6,Person_6,36,City_B,1707746162.9137092,ICEjjVXnhVmYCQ,Job_6
6,7,Person_7,37,City_B,1707746162.9137092,3q7aOu72hn3s2Q,Job_7
7,8,Person_8,38,City_B,1707746162.9137092,NmpTyT2bRNigUQ,Job_8


Sum of Ages 266


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)
# ANSWERS ABOVE

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX